# AI Fall 2022 - A4 - Decision Tree
## Your info

**Student Name:** _Your name_

**Student Id:** _Your std id_

## Practical Question - Decision Tree (25 + 5 points)
In this question you will first write a program to find an appropriate order of features for training a decision tree based on information gain. In the second part, you will use existing implementations to train and tune a classifier using GridSearch on the given dataset. And try to finish this in <15 mins.

### Import
Import the libraries you need in the next cell.

In [1]:
import math
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn import tree
import graphviz

# ...

### Part A (20 points)
Consider the following dataframe and answer the following questions:

In [2]:
df = pd.DataFrame({
    'Race': [1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    'Sex': [1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0],
    'Age': [43.0, 72.0, 4.0, 59.0, 34.0, 55.0, 41.0, 39.0, 6.0, 77.0, 67.0, 35.0, 45.0, 39.0, 32.0, 57.0, 99.0, 68.0, 47.0, 28.0],
    'BMI': [25.803, 33.394, 20.278, 32.918, 30.743, 37.734, 23.725, 20.722, 19.452, 22.044, 17.481, 18.918, 28.578, 19.214, 23.562, 23.446, 21.872, 20.691, 25.471, 25.82],
    'Smoker': [1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    'Label': [False, True, False, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, True, False],
})

df.head()

Race  Sex   Age     BMI  Smoker  Label
0   1.0  1.0  43.0  25.803     1.0  False
1   2.0  2.0  72.0  33.394     0.0   True
2   1.0  2.0   4.0  20.278     1.0  False
3   1.0  2.0  59.0  32.918     0.0  False
4   1.0  2.0  34.0  30.743     1.0  False

#### QA.1 (10 points)
Complete the following function that calculates and returns the list of features based on their information gain.

In [ ]:
def feature_order(X_data, y_data):
    '''
    Calculate the best feature order
    for training a decision tree based on
    information gain.
    Input:
        X_data (pd.DataFrame) contains data features
        y_data (pd.Series) contains the labels
    Output:
        order (list[str]): The `X_data` columns in the correct order
    '''
    order = list(X_data.columns)
    
    # Your code here
    
    # One could sort by implementing IG and entropy 
    # Or use DecisionTreeClassifier.feature_importances_.argsort
    
### Sample answer (Thanks to Mr. MohammadAmin Reisi)
    
    order = list(X_data.columns)
    
    # First calculate H(Y)
    h_y = entropy(y_data)
    # Now we compute IG(X) for every X
    values = X_data.values.reshape(-1,).tolist()
    info_gain = dict()
    for x in order:
        min_entropy = math.inf
        for threshold in values:
            new_X_data = X_data.copy()
            new_X_data[x] = new_X_data[x].apply(lambda value: 0 if value <= threshold else 1)
            groups = new_X_data.groupby(x)
            cur = 0.0
            for val, df in groups:
                prob = len(df.index) / len(X_data.index)
                cur = cur + prob * entropy(y_data.iloc[df.index,])
            min_entropy = min(min_entropy, cur)
        info_gain[x] = h_y - min_entropy
        print(x, min_entropy, info_gain[x])
    order.sort(reverse=True, key=lambda x : info_gain[x])

    return order
    
def entropy(y_data):
    val_counts = y_data.value_counts()
    val_sum = val_counts.sum()
    return val_counts.apply(lambda x : x / val_sum).apply(lambda x : -math.log2(x) * x).sum()

###

feature_order(df.iloc[:, :-1], df.Label)

#### QA.2 (5 points)
What are some of the effective approaches to prevent the tree from overfitting? (three approaches)

<font color=#183364>
    <i>
        Answer. Any answer including the following is acceptable:
        <ul>
            <li>Setting limits on leaves and depth</li>
            <li>Post-Pruning</li>
            <li>Ensemble</li>
            <li>Gradient Boosting</li>
            <li>...</li>
        </ul>
    </i>
</font>

#### QA.3 (5 points)
About the `Age` feature which is an integer-valued input attribute, how can a tree utilize it for classification? Write a small piece of code to find the answer for this specific column.

<font color=#183364>
    <i>
        Answer. We gotta sort the values and choose a threshold over their range to see when it yields the best information gain. In this case the best split is around: <pre> age=46 </pre>
    </i>
    
</font>

In [ ]:
### Sample answer (Thanks again to Mr. MohammadAmin Reisi)

X_data = df.iloc[:, :-1]
y_data = df.Label
h_y = entropy(y_data)

# Now we compute IG(X = "Age")
values = X_data["Age"].values.reshape(-1,).tolist()
best_threshold = -1
min_entropy = math.inf
for threshold in values:
    new_X_data = X_data.copy()
    new_X_data["Age"] = new_X_data["Age"].apply(lambda value: 0 if value <= threshold else 1)
    groups = new_X_data.groupby("Age")
    cur = 0.0
    for val, new_df in groups:
        prob = len(new_df.index) / len(X_data.index)
        cur = cur + prob * entropy(y_data.iloc[new_df.index,])
    if cur < min_entropy:
        min_entropy = cur
        best_threshold = threshold
###

best_threshold, min_entropy

### Part B (10 points)
In this part you will learn more about [sklearn.tree.DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) and use it to train a classifier and tune its parameters.

#### QB.1 (5 points)
For measuring the _accuracy_ of your models, choose a subset of the given dataset as your validation set (Check out [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)).

In [ ]:
# Your code here
## Split the dataset
TEST_SIZE = 0.3

X = df.drop('Label', axis=1)
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE)

In [ ]:
# Your code here
## Set your own hyperparameters below, such as:
dt_hyperparams = {
    'min_samples_split': 2,
    'criterion': 'entropy',
    'max_depth': 5,
    'min_samples_leaf': 4,
    'max_features': 4
}

## Train the model and check its performance
dt = DecisionTreeClassifier(**dt_hyperparams)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
accuracy_score(y_test, y_pred)

#### QB.2 (5 points)
Take a look at customizable and potentially useful input params of the model, define a range for each of their values, and use GridSearch (Feel free to implement/[import](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) it) and find a set of hyperparametetrs that work best among your candidates.

In [ ]:
# Your code here

hyperparams = {
    'criterion': ['entropy', 'gini', 'log_loss'],
    'max_depth': [1, 2, 5, 10],
    'min_samples_split': [2, 4, 8],
    'min_samples_leaf': [1, 2, 4, 8],
    'splitter': ['best', 'random'],
    'max_features': [2, 3, 4, 5]
}

clf = GridSearchCV(dt, hyperparams)
clf.fit(X_train, y_train)
clf.best_params_

#### QB.3 (5 extra points)
Visualize the tree splits and interpret it (Hint: Search about `IPython.display.Image`, `pydotplus`, `StringIO`, and `export_graphviz`).

In [ ]:
tree.plot_tree(
    dt,
    feature_names=df.columns,
    filled=True
)